### Movies for SSH and pressure - Aids to visualize the flow

In [1]:
#KRM

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.colors as mcolors

import matplotlib as mpl
#%matplotlib inline
from math import *

import scipy.io

import scipy as spy

from netCDF4 import Dataset

import pylab as pl

import os 

import sys

import seaborn as sns

In [2]:
lib_path = os.path.abspath('../../Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [3]:
CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run02/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run02/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

CanyonPressure='/ocean/kramosmu/MITgcm/TracerExperiments/3DDIFF/run02/phiHydGlob.nc'
CanyonPressureOut = Dataset(CanyonPressure)
#for dimobj in CanyonPressureOut.variables.values():
#    print dimobj

#CanyonPressureLow='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run07/phiHydLowGlob.nc'
#CanyonPressureLowOut = Dataset(CanyonPressureLow)
#for dimobj in CanyonPressureLowOut.variables.values():
#    print dimobj


In [4]:
# General input

nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
Time = CanyonStateOut.variables['T']

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells
yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

hFacW = rout.getField(CanyonGrid, 'HFacW') # (Z,Y,Xp1) for zonal fluxes
MaskW = rout.getMask(CanyonGrid,'HFacW') 

hFacS = rout.getField(CanyonGrid, 'HFacS') # (Z,Yp1,X) for meridional fluxes
MaskS = rout.getMask(CanyonGrid,'HFacS') 


In [5]:
# Load velocity components
uu = rout.getField(CanyonState,'U')
vv = rout.getField(CanyonState,'V')

U,V = rout.unstagger(uu,vv)



In [6]:
phiHyd = rout.getField(CanyonPressure,'phiHyd')
Eta = rout.getField(CanyonState,'Eta')

In [7]:
import matplotlib.animation as animation

In [8]:
sns.set()
sns.set_style('white')
sns.set_context("talk")
etamap = sns.cubehelix_palette(light=1, as_cmap=True)
presmap = sns.cubehelix_palette(start=0.3, rot=-0.5, as_cmap=True)

In [18]:
def Plot1(t,ax1):
    Uplot=np.ma.array(phiHyd[t,:,:,xslice],mask=MaskC[:,:,xslice])
    csU = np.linspace(PhiHydmin1,PhiHydmax1,num=31)
    csU2 = np.linspace(PhiHydmin1,PhiHydmax1,num=15)
    ax1.clear()
    mesh=ax1.contourf(yc[100:,200],z[:58],Uplot[:58,100:],csU,cmap=presmap)
    if t == 1:    
        cbar=plt.colorbar(mesh,ax=ax1,cmap ="RdYlGn_r")
    CS = ax1.contour(yc[100:,200],z[:58],Uplot[:58,100:],csU2,colors='k',linewidths=[0.75] )
    ax1.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax1.set_xlabel('Across-shore distance [km]')
    ax1.set_xticks([40000,50000,60000,70000,80000])
    ax1.set_xticklabels(['40', '50', '60', '70', '80'])
    ax1.set_ylabel('Depth [m]')
    ax1.set_title('Hydrostatic pressure pot. at day %0.1f [$m^2/s^2$]' %(t/2.0))

def Plot2(t,ax2):    
    Uplot=np.ma.array(phiHyd[t,:,yslice,:],mask=MaskC[:,yslice,:])
    csU = np.linspace(PhiHydmin2,PhiHydmax2,num=20)
    csU2 = np.linspace(PhiHydmin2,PhiHydmax2,num=11)
    ax2.clear()
    mesh=ax2.contourf(xc[200,100:260],z[:58],Uplot[:58,100:260],csU,cmap=presmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax2)
    CS=ax2.contour(xc[200,100:260],z[:58],Uplot[:58,100:260],csU2,colors='k',linewidths=[0.75] )
    ax2.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax2.set_xlabel('Alongshore distance [km]')
    ax2.set_xticks([50000,55000,60000,65000, 70000])
    ax2.set_xticklabels(['50','55','60','65','70'])
    ax2.set_ylabel('Depth [m]')
    ax2.set_title('Hydrostatic pressure pot. [$m^2/s^2$]')
    
def Plot3(t,ax3):   
    Etaplot = np.ma.array(Eta[t,:,:],mask=MaskC[0,:,:])
    uplane = np.ma.array(U[t,zslice2,:,1:],mask=MaskC[zslice2,1:,1:])
    vplane = np.ma.array(V[t,zslice2,1:,:],mask=MaskC[zslice2,1:,1:])
    csSpd = np.linspace(Etamin,Etamax,num=25)
    csU2 = np.linspace(Etamin,Etamax,num=16)
    ax3.clear()
    mesh=ax3.contourf(xc[145:265,:],yc[145:265,:],Etaplot[145:265,:],csSpd,cmap=etamap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax3)
    CS = ax3.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[145],colors='k',linewidths=[2.0] )
    vect = ax3.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20], vplane[145:265:5, ::20]
                      , color='k')
    
    MerPlane = ax3.plot(xc[yslice,:],yc[yslice,:],'k:',linewidth = 2)
    ZonPlane = ax3.plot(xc[145:265,xslice],yc[145:265,xslice],'k:',linewidth = 2)
    CS2=ax3.contour(xc[145:265,:],yc[145:265,:],Etaplot[145:265,:],csU2,colors='k',linewidths=[0.75])
    ax3.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax3.set_xlabel('Alongshore distance [km]')
    ax3.set_xticks([20000,40000,60000,80000,100000])
    ax3.set_xticklabels(['20', '40', '60', '80', '100'])
    ax3.set_ylabel('Across-shore distance [km]')
    ax3.set_yticks([40000,45000,50000,55000])
    ax3.set_yticklabels(['40', '45', '50', '55'])
    ax3.set_title('$\eta$ [$m$]')


def Plot4(t,ax4):
    
    Phiplot = np.ma.array(phiHyd[t,zslice,:,:],mask=MaskC[zslice,:,:])
    uplane = np.ma.array(U[t,zslice,:,1:],mask=MaskC[zslice,1:,1:])
    vplane = np.ma.array(V[t,zslice,1:,:],mask=MaskC[zslice,1:,1:])
    csSpd = np.linspace(PhiHydmin3,PhiHydmax3,num=30)
    csU2 = np.linspace(PhiHydmin3,PhiHydmax3,num=16)
    ax4.clear()
    mesh=ax4.contourf(xc[145:265,:],yc[145:265,:],Phiplot[145:265,:],csSpd,cmap=presmap)
    if t == 1:
        cbar=plt.colorbar(mesh,ax=ax4)
    CS = ax4.contour(xc[145:265,1:],yc[145:265,1:],bathy[145:265,1:],[145],colors='k',linewidths=[2.0] )
    CS2=ax4.contour(xc[145:265,:],yc[145:265,:],Phiplot[145:265,:],csU2,colors='k',linewidths=[0.75] )
    vect = ax4.quiver(xc[145:265:5, 1::20], yc[145:265:5, 1::20], uplane[145:265:5, ::20], vplane[145:265:5, ::20]
                      , color='k')
    
    MerPlane = ax4.plot(xc[yslice,:],yc[yslice,:],'k:',linewidth = 2)
    ZonPlane = ax4.plot(xc[145:265,xslice],yc[145:265,xslice],'k:',linewidth = 2)
    ax4.set_axis_bgcolor((205/255.0, 201/255.0, 201/255.0))
    ax4.set_xlabel('Alongshore distance [km]')
    ax4.set_xticks([20000,40000,60000,80000,100000])
    ax4.set_xticklabels(['20', '40', '60', '80', '100'])
    ax4.set_ylabel('Across-shore distance [km]')
    ax4.set_yticks([40000,45000,50000,55000])
    ax4.set_yticklabels(['40', '45', '50', '55'])
    ax4.set_title('Hydrostatic pressure pot. at shelf break')

In [22]:
## Animation
#N=5
xslice=180
yslice=235
zslice= 29 # shelf break index
zslice2=0 

PhiHydmax1 = 20
PhiHydmax2 = 16
PhiHydmax3 = 4.6

PhiHydmin1 = -1
PhiHydmin2 = -0.5
PhiHydmin3 = 3.6

Etamax = np.max(Eta)
Etamin = np.min(Eta)
#Empty figures
fig,((ax1,ax2),(ax3, ax4))  = plt.subplots(2, 2, figsize=(18, 12))

#Initial image
def init():
    Plot1(0,ax1)
    Plot2(0,ax2)
    Plot3(0,ax3)
    Plot4(0,ax4)
    
    
    plt.tight_layout()
    #return[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    

def animate(tt):
    Plot1(tt,ax1)
    Plot2(tt,ax2)
    Plot3(tt,ax3)
    Plot4(tt,ax4)
    
    plt.tight_layout()
#The animation function (max frames=47)   


anim = animation.FuncAnimation(fig, animate, init_func=init,frames=18, interval = 200,blit=False, repeat=False)

##A line that makes it all work
mywriter = animation.FFMpegWriter()

##Save in current folder
anim.save('Pressure_3DDIFF_run02.mp4',writer=mywriter,fps=0.2, dpi = 300, bitrate = 1000000, codec = "libx264")

plt.show()

